In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import cv2
import pickle
import tensorflow as tf
from keras import backend as K
import h5py

from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras import applications

with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/X_train_triplet.pkl', 'rb') as f:
    X = pickle.load(f)
    X = np.array(X)
#     X = np.expand_dims(X, axis=3)
    X = X/255.
with open('/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/y_train_triplet.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [0]:
print(np.shape(X), np.shape(y))


(235, 221, 221, 3) (235,)


In [0]:
def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


deep_rank_model = deep_rank_model()
deep_rank_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 221, 221, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [0]:
batch_size = 24

_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return K.maximum(loss, 0)


deep_rank_model.compile(loss=_loss_tensor, optimizer=SGD(lr=0.00001, momentum=0.9, nesterov=True))

def image_batch_generator(images, labels, batch_size):
    labels = np.array(labels)
    while True:
        batch_paths = np.random.choice(a = len(images), size = batch_size//3)
        input_1 = []
        
        for i in batch_paths:
            pos = np.where(labels == labels[i])[0]
            neg = np.where(labels != labels[i])[0]
            
            j = np.random.choice(pos)
            while j == i:
                j = np.random.choice(pos)
             
            k = np.random.choice(neg)
            while k == i:
                k = np.random.choice(neg)
            
            input_1.append(images[i])
            input_1.append(images[j])
            input_1.append(images[k])

        input_1 = np.array(input_1)
        input = [input_1, input_1, input_1]
        yield(input, np.zeros((batch_size, )))

from keras.callbacks import ModelCheckpoint

filepath = '/content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/triplet_weight.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

print("Traning.....")
deep_rank_model.fit_generator(generator=image_batch_generator(X, y, batch_size),
                   steps_per_epoch=len(X)//batch_size,
                   epochs=30,
                   verbose=1,
                   callbacks=callbacks_list)
print("finished!...")

Traning.....
Epoch 1/30
9/9 [==============================] - 11s 1s/step - loss: 0.0152

Epoch 00001: loss improved from inf to 0.01519, saving model to /content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/triplet_weight.hdf5
Epoch 2/30
9/9 [==============================] - 7s 778ms/step - loss: 0.0305

Epoch 00002: loss did not improve from 0.01519
Epoch 3/30
9/9 [==============================] - 7s 782ms/step - loss: 0.0137

Epoch 00003: loss improved from 0.01519 to 0.01365, saving model to /content/gdrive/My Drive/app/FacialRecognition/FaceNet/new1/triplet_weight.hdf5
Epoch 4/30
9/9 [==============================] - 7s 783ms/step - loss: 0.0770

Epoch 00004: loss did not improve from 0.01365
Epoch 5/30
2/9 [=====>........................] - ETA: 5s - loss: 0.0452    

KeyboardInterrupt: ignored